# Senate Bill Status

> In the current legislative status field as well as the Legislative history section, there are free text strings that depic actions being taken on these bills, their dates, and the potential actors. However, they are not particularly well-formatted, so we'll need to do some custom string parsing to get this to work.


In [3]:
# | default_exp senate.bill_status
# | export
import re
import datetime

from typing import List
from nbdev.showdoc import show_doc

from legisph.senate.models import SenateBill, Senator, SenateCommittee

## Parsing Strategy 

Our strategy is to cycle through a predefined list of classes that all have a `parse()` class method which return parsed statuses (if any) and an indicator as to whether to short circuit the function. 

### Pending In Committee

For example, we first define the most common status for "Pending in committee" as follows:

In [4]:
# | export
class PendingInCommittee(SenateBill.SenateBillStatus):
    name: str = "Pending in Committee"

    @classmethod
    def parse(cls, h):
        if h.item == "Pending in the Committee":
            return (cls(**h.dict()), False)
        return (None, True)

This is able to parse a status and turn it into the relevant subclass. It returns `False` for the second part of the tuple because if it matches exactly, then there is no further action to take. We also define a convenience function for testing the individual parsing functions:

In [148]:
def test_parsing(statusclass, item):
    parsed = statusclass.parse(
        SenateBill.SenateBillStatus(date=datetime.date.today(), item=item)
    )
    assert parsed[0] is not None
    assert isinstance(parsed[1], bool)
    return parsed

In [149]:
test_parsing(PendingInCommittee, "Pending in the Committee")

(PendingInCommittee(date=datetime.date(2022, 10, 10), item='Pending in the Committee', name='Pending in Committee'),
 False)

This function below then takes a predefined list of classes and cycles through them, chugging out the parsed actions along the way.

In [6]:
# | export
def parse_senate_bill_status(
    status: SenateBill.SenateBillStatus,  # Senate Bill Status to parse into a subclass
    classes: list,  # List of classes through which to cycle
):
    actions = []
    for c in classes:
        action, cycle = c.parse(status)
        if action is not None:
            actions.append(action)
        if not cycle:
            break
    return actions


show_doc(parse_senate_bill_status)

---

### parse_senate_bill_status

>      parse_senate_bill_status
>                                (status:legisph.senate.models.SenateBill.Senate
>                                BillStatus, classes:list)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| status | SenateBillStatus | Senate Bill Status to parse into a subclass |
| classes | list | List of classes through which to cycle |

In [7]:
parse_senate_bill_status(
    SenateBill.SenateBillStatus(
        date=datetime.date(2022, 10, 7), item="Pending in the Committee"
    ),
    classes=[PendingInCommittee],
)

[PendingInCommittee(date=datetime.date(2022, 10, 7), item='Pending in the Committee', name='Pending in Committee')]

## Parsing All Statuses

We then proceed to define all the remaining statuses below:


### Joint Proceedings

In [8]:
# | export
class JointProceedings(SenateBill.SenateBillStatus):
    name: str = "Conducted Joint Proceedings"

    @classmethod
    def parse(cls, h):
        if h.item == "Conducted JOINT COMMITTEE MEETINGS/HEARINGS;":
            return (cls(**h.dict()), False)
        return (None, True)

In [46]:
test_parsing(JointProceedings, "Conducted JOINT COMMITTEE MEETINGS/HEARINGS;")

(JointProceedings(date=datetime.date(2022, 10, 10), item='Conducted JOINT COMMITTEE MEETINGS/HEARINGS;', name='Conducted Joint Proceedings'),
 False)

### Introduced

In [10]:
# | export
class Introduced(SenateBill.SenateBillStatus):
    name: str = "Introduced by a Senator"
    senator: Senator

    @classmethod
    def parse(cls, h):
        if h.item.startswith("Introduced by Senator "):
            return (
                cls(
                    **h.dict(),
                    senator=Senator(
                        name=(
                            h.item.replace("Introduced by Senator ", "").replace(
                                ";", ""
                            )
                        )
                    )
                ),
                True,
            )
        return (None, True)

In [49]:
test_parsing(Introduced, "Introduced by Senator JINGGOY P. EJERCITO-ESTRADA;")

(Introduced(date=datetime.date(2022, 10, 10), item='Introduced by Senator JINGGOY P. EJERCITO-ESTRADA;', name='Introduced by a Senator', senator=Senator(name='JINGGOY P. EJERCITO-ESTRADA')),
 True)

### Committee Report Calendared For Ordinary Business

In [124]:
# | export
class CommitteeReportCalendaredForOrdinaryBusiness(SenateBill.SenateBillStatus):
    name: str = "Committe Report Calendared for Ordinary Business"

    @classmethod
    def parse(cls, h):
        if h.item in (
            "Committee Report Calendared for Ordinary Business;",
            "Committee Report calendared for Ordinary Business;",
        ):
            return (cls(**h.dict()), False)
        return (None, True)

In [51]:
test_parsing(
    CommitteeReportCalendaredForOrdinaryBusiness,
    "Committee Report Calendared for Ordinary Business;",
)

(CommitteeReportCalendaredForOrdinaryBusiness(date=datetime.date(2022, 10, 10), item='Committee Report Calendared for Ordinary Business;', name='Committe Report Calendared for Ordinary Business'),
 False)

### Consolidated or Substituted in Committee Report

In [14]:
# | export
class ConsolidatedOrSubstitutedInCommitteeReport(SenateBill.SenateBillStatus):
    name: str = "Consolidated or Substituted in Committee Report"

    @classmethod
    def parse(cls, h):
        if h.item == "Consolidated/Substituted in the Committee Report":
            return (cls(**h.dict()), False)
        return (None, True)

In [56]:
test_parsing(
    ConsolidatedOrSubstitutedInCommitteeReport,
    "Consolidated/Substituted in the Committee Report",
)

(ConsolidatedOrSubstitutedInCommitteeReport(date=datetime.date(2022, 10, 10), item='Consolidated/Substituted in the Committee Report', name='Consolidated or Substituted in Committee Report'),
 False)

### Technical Working Group

In [16]:
# | export
class TechnicalWorkingGroup(SenateBill.SenateBillStatus):
    name: str = "Conducted a Technical Working Group"

    @classmethod
    def parse(cls, h):
        if h.item == "Conducted TECHNICAL WORKING GROUP;":
            return (cls(**h.dict()), False)
        return (None, True)

In [57]:
test_parsing(TechnicalWorkingGroup, "Conducted TECHNICAL WORKING GROUP;")

(TechnicalWorkingGroup(date=datetime.date(2022, 10, 10), item='Conducted TECHNICAL WORKING GROUP;', name='Conducted a Technical Working Group'),
 False)

### First Reading

In [34]:
# | export
class FirstReading(SenateBill.SenateBillStatus):
    name: str = "Read on First Reading and Referred to Committee"
    committees: List[SenateCommittee]

    @classmethod
    def parse(cls, h):
        slug1 = "Read on First Reading and Referred to the Committee on "
        slug2 = "Read on First Reading and Referred to the Committee(s) on "
        slug3 = "Read on First Reading and referred to the Committee(s) on "
        if h.item.startswith(slug1):
            committee = SenateCommittee(name=h.item.replace(slug1, "").replace(";", ""))
            return (cls(**h.dict(), committees=[committee]), True)
        if h.item.startswith(slug2) or h.item.startswith(slug3):
            committees = h.item.replace(slug2, "")
            committees = h.item.replace(slug3, "")
            committees = re.split("[ ]*and[ ]*|[ ]*;[ ]*", committees)
            committees = [
                SenateCommittee(name=name) for name in committees if name != ""
            ]
            return (cls(**h.dict(), committees=committees), True)
        return (None, True)

In [58]:
test_parsing(
    FirstReading,
    "Read on First Reading and Referred to the Committee on JUSTICE AND HUMAN RIGHTS;",
)

(FirstReading(date=datetime.date(2022, 10, 10), item='Read on First Reading and Referred to the Committee on JUSTICE AND HUMAN RIGHTS;', name='Read on First Reading and Referred to Committee', committees=[SenateCommittee(name='JUSTICE AND HUMAN RIGHTS')]),
 False)

In [59]:
test_parsing(
    FirstReading,
    "Read on First Reading and Referred to the Committee(s) on EDUCATION, ARTS AND CULTURE; and FINANCE;",
)

(FirstReading(date=datetime.date(2022, 10, 10), item='Read on First Reading and Referred to the Committee(s) on EDUCATION, ARTS AND CULTURE; and FINANCE;', name='Read on First Reading and Referred to Committee', committees=[SenateCommittee(name='Read on First Reading'), SenateCommittee(name='Referred to the Committee(s) on EDUCATION, ARTS AND CULTURE'), SenateCommittee(name='FINANCE')]),
 False)

In [60]:
test_parsing(
    FirstReading,
    "Read on First Reading and Referred to the Committee(s) on LOCAL GOVERNMENT; CIVIL SERVICE, GOVERNMENT REORGANIZATION AND PROFESSIONAL REGULATION and FINANCE;",
)

(FirstReading(date=datetime.date(2022, 10, 10), item='Read on First Reading and Referred to the Committee(s) on LOCAL GOVERNMENT; CIVIL SERVICE, GOVERNMENT REORGANIZATION AND PROFESSIONAL REGULATION and FINANCE;', name='Read on First Reading and Referred to Committee', committees=[SenateCommittee(name='Read on First Reading'), SenateCommittee(name='Referred to the Committee(s) on LOCAL GOVERNMENT'), SenateCommittee(name='CIVIL SERVICE, GOVERNMENT REORGANIZATION AND PROFESSIONAL REGULATION'), SenateCommittee(name='FINANCE')]),
 False)

In [61]:
test_parsing(
    FirstReading,
    "Read on First Reading and referred to the Committee(s) on EDUCATION, ARTS AND CULTURE; and FINANCE;",
)

(FirstReading(date=datetime.date(2022, 10, 10), item='Read on First Reading and referred to the Committee(s) on EDUCATION, ARTS AND CULTURE; and FINANCE;', name='Read on First Reading and Referred to Committee', committees=[SenateCommittee(name='EDUCATION, ARTS AND CULTURE'), SenateCommittee(name='FINANCE')]),
 False)

### Committee Proceedings

In [22]:
# | export
class CommitteeProceedings(SenateBill.SenateBillStatus):
    name: str = "Conducted Committee Proceedings"

    @classmethod
    def parse(cls, h):
        if h.item == "Conducted COMMITTEE MEETINGS/HEARINGS;":
            return (cls(**h.dict()), False)
        return (None, True)

In [62]:
test_parsing(CommitteeProceedings, "Conducted COMMITTEE MEETINGS/HEARINGS;")

(CommitteeProceedings(date=datetime.date(2022, 10, 10), item='Conducted COMMITTEE MEETINGS/HEARINGS;', name='Conducted Committee Proceedings'),
 False)

### Approved on Second Reading

In [24]:
# | export
class ApprovedOnSecondReading(SenateBill.SenateBillStatus):
    name: str = "Approved On Second Reading"
    with_amendments: bool

    @classmethod
    def parse(cls, h):
        if h.item == "Approved on Second Reading with Amendments;":
            return (cls(**h.dict(), with_amendments=True), False)
        if h.item == "Approved on Second Reading without Amendment;":
            return (cls(**h.dict(), with_amendments=False), False)
        return (None, True)

In [63]:
test_parsing(ApprovedOnSecondReading, "Approved on Second Reading with Amendments;")

(ApprovedOnSecondReading(date=datetime.date(2022, 10, 10), item='Approved on Second Reading with Amendments;', name='Approved On Second Reading', with_amendments=True),
 False)

In [64]:
test_parsing(ApprovedOnSecondReading, "Approved on Second Reading without Amendment;")

(ApprovedOnSecondReading(date=datetime.date(2022, 10, 10), item='Approved on Second Reading without Amendment;', name='Approved On Second Reading', with_amendments=False),
 False)

### Transferred to Calendar for Special Order

In [27]:
# | export
class TransferredToCalendarForSpecialOrder(SenateBill.SenateBillStatus):
    name: str = "Transferred to Calendar for Special Order"

    @classmethod
    def parse(cls, h):
        if (
            h.item
            == "Transferred from the Calendar for Ordinary Business to the Calendar for Special Order;"
        ):
            return (cls(**h.dict()), False)
        return (None, True)

In [65]:
test_parsing(
    TransferredToCalendarForSpecialOrder,
    "Transferred from the Calendar for Ordinary Business to the Calendar for Special Order;",
)

(TransferredToCalendarForSpecialOrder(date=datetime.date(2022, 10, 10), item='Transferred from the Calendar for Ordinary Business to the Calendar for Special Order;', name='Transferred to Calendar for Special Order'),
 False)

### Period of Interpellation Closed

In [66]:
# | export
class PeriodOfInterpellationClosed(SenateBill.SenateBillStatus):
    name: str = "Period of Interpellation Closed"

    @classmethod
    def parse(cls, h):
        if h.item == "Period of interpellation closed;":
            return (cls(**h.dict()), False)
        return (None, True)

In [68]:
test_parsing(PeriodOfInterpellationClosed, "Period of interpellation closed;")

(PeriodOfInterpellationClosed(date=datetime.date(2022, 10, 10), item='Period of interpellation closed;', name='Period of Interpellation Closed'),
 False)

### Sponsorship Speech

In [152]:
# | export
class SponsorshipSpeech(SenateBill.SenateBillStatus):
    name: str = "Sponsorship Speech"
    senators: List[Senator]

    @classmethod
    def parse(cls, h):
        slug1 = "Sponsorship speech of Senator(s) "
        if h.item.startswith(slug1):
            senators = (
                h.item.replace(slug1, "")
                .replace(" on the Conference Committee Report;", "")
                .replace(" on the conference committee report;", "")
            )
            senators = [Senator(name=name) for name in senators.split(" and ")]
            return (cls(**h.dict(), senators=senators), True)
        slug2 = "Sponsorship speech on the Conference Committee Report of Senator "
        if h.item.startswith(slug2):
            senators = [Senator(name=h.item.replace(slug2, "").replace(";", ""))]
            return (cls(**h.dict(), senators=senators), True)
        slug3 = "Sponsorship speech on the conference committee report of Senator "
        if h.item.startswith(slug3):
            senators = [Senator(name=h.item.replace(slug3, "").replace(";", ""))]
            return (cls(**h.dict(), senators=senators), True)
        slug4 = "Sponsorship speech of Senator "
        if h.item.startswith(slug4):
            senators = [
                Senator(
                    name=(
                        h.item.replace(slug4, "")
                        .replace(";", "")
                        .replace(" on the conference committee report", "")
                        .replace(" on the Conference Committee Report", "")
                    )
                )
            ]
            return (cls(**h.dict(), senators=senators), True)
        slug5 = "Sponsorship speech delivered by Senator "
        if h.item.startswith(slug5):
            senators = [Senator(name=h.item.replace(slug5, "").replace(";", ""))]
            return (cls(**h.dict(), senators=senators), True)
        slug6 = "Sponsorship Speech on the Conference Committee Report of Senator "
        slug7 = "Sponsorship speech on the  Conference Committee Report of Senator "
        if h.item.startswith(slug6) or h.item.startswith(slug7):
            senators = [
                Senator(
                    name=(h.item.replace(slug6, "").replace(slug7, "").replace(";", ""))
                )
            ]
            return (cls(**h.dict(), senators=senators), True)
        return (None, True)

In [75]:
test_parsing(
    SponsorshipSpeech,
    "Sponsorship speech of Senator(s) BENIGNO S. AQUINO III and JINGGOY P. EJERCITO-ESTRADA;",
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 10), item='Sponsorship speech of Senator(s) BENIGNO S. AQUINO III and JINGGOY P. EJERCITO-ESTRADA;', name='Sponsorship Speech', senators=[Senator(name='BENIGNO S. AQUINO III'), Senator(name='JINGGOY P. EJERCITO-ESTRADA;')]),
 True)

In [80]:
test_parsing(
    SponsorshipSpeech,
    "Sponsorship speech of Senator(s) EDGARDO J. ANGARA on the Conference Committee Report;",
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 10), item='Sponsorship speech of Senator(s) EDGARDO J. ANGARA on the Conference Committee Report;', name='Sponsorship Speech', senators=[Senator(name='EDGARDO J. ANGARA')]),
 True)

In [94]:
test_parsing(
    SponsorshipSpeech,
    "Sponsorship speech on the Conference Committee Report of Senator JOEL VILLANUEVA;",
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 10), item='Sponsorship speech on the Conference Committee Report of Senator JOEL VILLANUEVA;', name='Sponsorship Speech', senators=[Senator(name='JOEL VILLANUEVA')]),
 True)

In [102]:
test_parsing(
    SponsorshipSpeech,
    'Sponsorship speech on the conference committee report of Senator EMMANUEL "MANNY" D. PACQUIAO;',
)

(None, True)

In [107]:
test_parsing(
    SponsorshipSpeech,
    'Sponsorship speech of Senator RONALD "BATO" DELA ROSA on the conference committee report;',
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 10), item='Sponsorship speech of Senator RONALD "BATO" DELA ROSA on the conference committee report;', name='Sponsorship Speech', senators=[Senator(name='RONALD "BATO" DELA ROSA')]),
 True)

In [109]:
test_parsing(
    SponsorshipSpeech,
    "Sponsorship speech delivered by Senator SONNY ANGARA;",
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 10), item='Sponsorship speech delivered by Senator SONNY ANGARA;', name='Sponsorship Speech', senators=[Senator(name='SONNY ANGARA')]),
 True)

In [116]:
test_parsing(
    SponsorshipSpeech,
    "Sponsorship Speech on the Conference Committee Report of Senator RISA HONTIVEROS;",
)

(SponsorshipSpeech(date=datetime.date(2022, 10, 10), item='Sponsorship Speech on the Conference Committee Report of Senator RISA HONTIVEROS;', name='Sponsorship Speech', senators=[Senator(name='RISA HONTIVEROS')]),
 True)

### Sponsorship Speech Inserted into Record

In [135]:
# | export
class SponsorshipSpeechInsertedIntoRecord(SenateBill.SenateBillStatus):
    name: str = "Sponsorship Speech Inserted into Record"

    @classmethod
    def parse(cls, h):
        if h.item == "Sponsorship speech inserted into the record;":
            return (cls(**h.dict()), False)
        return (None, True)

In [136]:
test_parsing(
    SponsorshipSpeechInsertedIntoRecord,
    "Sponsorship speech inserted into the record;",
)

(SponsorshipSpeechInsertedIntoRecord(date=datetime.date(2022, 10, 10), item='Sponsorship speech inserted into the record;', name='Sponsorship Speech Inserted into Record'),
 False)

### Approved on Third Reading

In [137]:
# | export
class ApprovedOnThirdReading(SenateBill.SenateBillStatus):
    name: str = "Approved on Third Reading"

    @classmethod
    def parse(cls, h):
        if h.item == "Approved on Third Reading;":
            return (cls(**h.dict()), False)
        return (None, True)

In [127]:
test_parsing(
    ApprovedOnThirdReading,
    "Approved on Third Reading;",
)

(ApprovedOnThirdReading(date=datetime.date(2022, 10, 10), item='Approved on Third Reading;', name='Approved on Third Reading'),
 False)

### Sent to the House of Representatives

In [132]:
# | export
class SentToHouseOfRepresentatives(SenateBill.SenateBillStatus):
    name: str = "Sent to the House of Representatives"

    @classmethod
    def parse(cls, h):
        if h.item == "Sent to the House of Representatives requesting for concurrence;":
            return (cls(**h.dict()), False)
        return (None, True)

In [134]:
test_parsing(
    SentToHouseOfRepresentatives,
    "Sent to the House of Representatives requesting for concurrence;",
)

(SentToHouseOfRepresentatives(date=datetime.date(2022, 10, 10), item='Sent to the House of Representatives requesting for concurrence;', name='Sent to the House of Representatives'),
 False)

### Individual amendmends opened

In [153]:
# | export
class IndividualAmendmentsOpened(SenateBill.SenateBillStatus):
    name: str = "Individual amendments opened"

    @classmethod
    def parse(cls, h):
        if h.item == "Period of individual amendments;":
            return (cls(**h.dict()), False)
        return (None, True)

In [154]:
test_parsing(
    IndividualAmendmentsOpened,
    "Period of individual amendments;",
)

(IndividualAmendmentsOpened(date=datetime.date(2022, 10, 10), item='Period of individual amendments;', name='Individual amendments opened'),
 False)

### Individual amendments closed

In [138]:
# | export
class IndividualAmendmentsClosed(SenateBill.SenateBillStatus):
    name: str = "Individual amendments closed"

    @classmethod
    def parse(cls, h):
        if h.item == "Period of individual amendments closed;":
            return (cls(**h.dict()), False)
        return (None, True)

In [141]:
test_parsing(
    IndividualAmendmentsClosed,
    "Period of individual amendments closed;",
)

(IndividualAmendmentsClosed(date=datetime.date(2022, 10, 10), item='Period of individual amendments closed;', name='Individual amendments closed'),
 False)

### Committee amendmends opened

In [161]:
# | export
class CommitteeAmendmentsOpened(SenateBill.SenateBillStatus):
    name: str = "Committee amendments opened"

    @classmethod
    def parse(cls, h):
        if h.item == "Period of committee amendments;":
            return (cls(**h.dict()), False)
        return (None, True)

In [162]:
test_parsing(
    CommitteeAmendmentsOpened,
    "Period of committee amendments;",
)

(CommitteeAmendmentsOpened(date=datetime.date(2022, 10, 10), item='Period of committee amendments;', name='Committee amendments opened'),
 False)

### Committee amendments closed

In [143]:
# | export
class CommitteeAmendmentsClosed(SenateBill.SenateBillStatus):
    name: str = "Committee amendments closed"

    @classmethod
    def parse(cls, h):
        if h.item == "Period of committee amendments closed;":
            return (cls(**h.dict()), False)
        return (None, True)

In [150]:
test_parsing(
    CommitteeAmendmentsClosed,
    "Period of committee amendments closed;",
)

(CommitteeAmendmentsClosed(date=datetime.date(2022, 10, 10), item='Period of committee amendments closed;', name='Committee amendments closed'),
 False)

### Printed copies distributed to Senators

In [155]:
# | export
class CopiesDistributed(SenateBill.SenateBillStatus):
    name: str = "Copies Distributed to Senators"

    @classmethod
    def parse(cls, h):
        if h.item == "Printed copies were distributed to the Senators;":
            return (cls(**h.dict()), False)
        return (None, True)

In [156]:
test_parsing(
    CopiesDistributed,
    "Printed copies were distributed to the Senators;",
)

(CopiesDistributed(date=datetime.date(2022, 10, 10), item='Printed copies were distributed to the Senators;', name='Copies Distributed to Senators'),
 False)

### Consolidated with Approved Bill

In [164]:
# | export
class ConsolidatedWithApprovedBill(SenateBill.SenateBillStatus):
    name: str = "Consolidated with Approved Bill"

    @classmethod
    def parse(cls, h):
        if h.item == "Consolidated with Approved Bill":
            return (cls(**h.dict()), False)
        return (None, True)

In [165]:
test_parsing(
    ConsolidatedWithApprovedBill,
    "Consolidated with Approved Bill",
)

(ConsolidatedWithApprovedBill(date=datetime.date(2022, 10, 10), item='Consolidated with Approved Bill', name='Consolidated with Approved Bill'),
 False)

### Approved by the President

In [160]:
# | export
class ApprovedByPresident(SenateBill.SenateBillStatus):
    name: str = "Approved by the President"

    @classmethod
    def parse(cls, h):
        if h.item == "Approved by the President of the Philippines":
            return (cls(**h.dict()), False)
        return (None, True)

In [159]:
test_parsing(
    ApprovedByPresident,
    "Approved by the President of the Philippines",
)

(ApprovedByPresident(date=datetime.date(2022, 10, 10), item='Approved by the President of the Philippines', name='Approved by the President'),
 False)